In [1]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt 
import seaborn as sns

## GroupBy Mechanics

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'], 'key2': ['one', 'two', 'one', 'two', 'one'],
                 'data1': np.random.randn(5), 'data2': np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,1.181185,-1.946683
1,a,two,-0.059175,-0.692519
2,b,one,0.991436,-0.727444
3,b,two,-0.444959,-0.649992
4,a,one,0.909826,-0.421777


In [8]:
grouped = df['data1'].groupby(df['key1'])

In [9]:
grouped

In [10]:
# to compute group means we can call the GroupBy’s mean method:
grouped.mean()

key1
a   -0.205009
b    0.942852
Name: data1, dtype: float64

In [11]:
# If instead we had passed multiple arrays as a list, we’d get something different:

means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [12]:
means

key1  key2
a     one    -0.140396
      two    -0.334235
b     one     1.001669
      two     0.884036
Name: data1, dtype: float64

In [13]:
means.unstack()

key2,one,two
key1,,
a,-0.140396,-0.334235
b,1.001669,0.884036


In [3]:
# In this example, the group keys are all Series, though they could be any arrays of the right length:

states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years  = np.array([2005, 2005, 2006, 2005, 2006])



In [18]:
df['data1'].groupby([states, years]).mean()

California  2005   -0.334235
            2006    1.001669
Ohio        2005    0.137753
            2006    0.327736
Name: data1, dtype: float64

In [19]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.205009,1.023758
b,0.942852,-0.642827


In [20]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.140396  0.638177
     two  -0.334235  1.794918
b    one   1.001669 -1.357181
     two   0.884036  0.071528

In [21]:
# Regardless of the objective in using groupby, a generally useful GroupBy method is
# size, which returns a Series containing group sizes:

df.groupby(['key1', 'key2']).size()


key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## Iterating Over Groups

In [23]:
# GroupBy object supports iteration, generating a sequence of 2-tuples containing
# the group name along with the chunk of data

for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.608529  0.489209
1    a  two -0.334235  1.794918
4    a  one  0.327736  0.787146
b
  key1 key2     data1     data2
2    b  one  1.001669 -1.357181
3    b  two  0.884036  0.071528


In [25]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)


('a', 'one')
  key1 key2     data1     data2
0    a  one -0.608529  0.489209
4    a  one  0.327736  0.787146
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.334235  1.794918
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.001669 -1.357181
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.884036  0.071528


In [26]:
pieces = dict(list(df.groupby('key1')))

In [27]:
pieces['b']

,key1,key2,data1,data2
2,b,one,1.001669,-1.357181
3,b,two,0.884036,0.071528


In [29]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [31]:
for dtype, group in grouped:
    print(dtype)
    print(group)

a
0   -0.608529
1   -0.334235
4    0.327736
Name: data1, dtype: float64
b
2    1.001669
3    0.884036
Name: data1, dtype: float64


### Selecting a Column or Subset of Columns


In [ ]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

are syntactic sugar for:
    
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [37]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.638177
     two   1.794918
b    one  -1.357181
     two   0.071528

In [40]:
# if a list or array is passed or a grouped Series if only a single column name is passed as a scalar:

s_grouped = df.groupby(['key1', 'key2'])['data2']

s_grouped

In [43]:

s_grouped.mean()

key1  key2
a     one     0.638177
      two     1.794918
b     one    -1.357181
      two     0.071528
Name: data2, dtype: float64

## Grouping with Dicts and Series


In [45]:
people = pd.DataFrame(np.random.randn(5, 5), 
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1,2]] = np.nan

In [46]:
people

,a,b,c,d,e
Joe,-0.393261,0.416812,-0.866676,0.949681,-0.469459
Steve,0.139420,0.650309,1.106556,-1.059255,0.027751
Wes,1.327820,NaN,NaN,1.620991,0.093204
Jim,-0.174133,0.174827,-2.118513,1.099214,-1.006149
Travis,0.868983,2.606475,-2.490078,0.073856,-0.412476


In [49]:
# Now, suppose I have a group correspondence for the columns and want to sum
# together the columns by group:

mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 
           'd': 'blue', 'e': 'red', 'f': 'orange'}

# Now, you could construct an array from this dict to pass to groupby, but instead we
# can just pass the dict (I included the key 'f' to highlight that unused grouping keys are OK)

by_column = people.groupby(mapping, axis=1)

by_column.sum()

,blue,red
Joe,0.083005,-0.445908
Steve,0.047301,0.817480
Wes,1.620991,1.421024
Jim,-1.019299,-1.005455
Travis,-2.416222,3.062982


In [51]:
# The same functionality holds  for Series, which can be viewed as a fixed-size mapping

map_series = pd.Series(mapping)

map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [54]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## Grouping with Functions

In [58]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.760425,0.591639,-2.985189,3.669887,-1.382403
5,0.139420,0.650309,1.106556,-1.059255,0.027751
6,0.868983,2.606475,-2.490078,0.073856,-0.412476


In [56]:
# Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally
key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.393261  0.416812 -0.866676  0.949681 -0.469459
  two -0.174133  0.174827 -2.118513  1.099214 -1.006149
5 one  0.139420  0.650309  1.106556 -1.059255  0.027751
6 two  0.868983  2.606475 -2.490078  0.073856 -0.412476

### Grouping by Index Levels

In [61]:
columns = pd.MultiIndex.from_arrays([['US','US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]], names=['city', 'tenor'])

hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

hier_df


city         US                            JP          
tenor         1         3         5         1         3
0     -1.047835  0.393007 -0.417770  0.096509  0.690401
1     -0.158161  0.704055 -1.788438 -1.751751 -1.060424
2      1.112738  1.127971 -0.902601  0.111239  1.116364
3      0.837047  1.135288 -2.306473  1.003301  0.098904

In [62]:
# To group by level, pass the level number or name using the level keyword

hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


### Data Aggregation

In [ ]:
# Aggregations refer to any data transformation that produces scalar values from arrays.

In [63]:
df

,key1,key2,data1,data2
0,a,one,-0.608529,0.489209
1,a,two,-0.334235,1.794918
2,b,one,1.001669,-1.357181
3,b,two,0.884036,0.071528
4,a,one,0.327736,0.787146


In [65]:
grouped = df.groupby('key1')

In [66]:
grouped['data1'].quantile(0.9)

key1
a    0.195342
b    0.989905
Name: data1, dtype: float64

In [9]:
#To use your own aggregation functions, pass any function that aggregates an array to the aggregate or agg method
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [68]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.936265,1.305708
b,0.117633,1.428709


In [69]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.205009  0.481324 -0.608529 -0.471382 -0.334235 -0.003249   
b      2.0  0.942852  0.083179  0.884036  0.913444  0.942852  0.972260   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.327736   3.0  1.023758  0.684257  0.489209  0.638177  0.787146   
b     1.001669   2.0 -0.642827  1.010250 -1.357181 -1.000004 -0.642827   

                          
           75%       max  
key1                      
a     1.291032  1.794918  
b    -0.285649  0.071528

### Column-Wise and Multiple Function Application


In [19]:
tips = pd.read_csv('C:/Users/Justin/Downloads/pydata-book-2nd-edition/examples/tips.csv')

In [20]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [4]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [5]:
grouped = tips.groupby(['day', 'smoker'])

In [6]:
grouped_pct = grouped['tip_pct']

In [7]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [10]:
# If you pass a list of functions or function names instead, you get back a DataFrame
# with column names taken from the functions

grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [11]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [12]:
functions = ['count', 'mean', 'max']

result = grouped['tip_pct', 'total_bill'].agg(functions)

<ipython-input-12-9133894e0c40>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


In [83]:
result


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [13]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [14]:
# As before, a list of tuples with custom names can be passed:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

grouped['tip_pct','total_bill'].agg(ftuples)

<ipython-input-14-1f721a8b1b14>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct','total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [15]:
# pass a dict to agg that contains a mapping of column names
# to any of the function specifications 
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [88]:
grouped.agg({'tip_pct': ['min', 'max','mean', 'std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

#### Returning Aggregated Data Without Row Indexes


In [16]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


 ### Apply: General split-apply-combine


In [18]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [19]:
# if we group by smoker, say, and call apply with this function, we get the following:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [20]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [23]:
# You may recall that I earlier called describe on a GroupBy object:
result = tips.groupby('smoker')['tip_pct'].describe()

result


,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [24]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [ ]:
# Inside GroupBy, when you invoke a method like describe, it is actually just a short‐
# cut for:
f = lambda x: x.describe()
grouped.apply(f)


## Suppressing the Group Keys

In [25]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

In [28]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

quartiles = pd.cut(frame.data1, 4)

quartiles[:10]

0    (-1.646, 0.148]
1    (-1.646, 0.148]
2    (-1.646, 0.148]
3     (0.148, 1.943]
4     (0.148, 1.943]
5    (-1.646, 0.148]
6    (-1.646, 0.148]
7    (-1.646, 0.148]
8    (-1.646, 0.148]
9     (1.943, 3.737]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.447, -1.646] < (-1.646, 0.148] < (0.148, 1.943] < (1.943, 3.737]]

In [31]:
# The Categorical object returned by cut can be passed directly to groupby. So we
# could compute a set of statistics for the data2 column like so:
def get_stats(group):
    return{'min': group.min(), 'max': group.max(),
          'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)


grouped.apply(get_stats).unstack()


,min,max,count,mean
data1,,,,
"(-3.447, -1.646]",-1.770554,2.550293,56.0,0.006346
"(-1.646, 0.148]",-2.694127,3.006283,518.0,0.012870
"(0.148, 1.943]",-3.230947,2.811046,389.0,0.107990
"(1.943, 3.737]",-2.061038,2.319826,37.0,-0.075258


In [32]:
# These were equal-length buckets; to compute equal-size buckets based on sample
# quantiles, use qcut. I’ll pass labels=False to just get quantile numbers:

# Return quantile numbers

grouping = pd.qcut(frame.data1, 10, labels=False)

grouped = frame.data2.groupby(grouping)

grouped.apply(get_stats).unstack()


,min,max,count,mean
data1,,,,
0,-2.694127,2.550293,100.0,-0.092416
1,-2.259282,2.891289,100.0,0.218802
2,-2.254857,2.405751,100.0,0.184846
3,-2.331866,2.380809,100.0,-0.104367
4,-2.237706,2.346130,100.0,-0.018569
5,-2.004042,3.006283,100.0,-0.093301
6,-2.963645,2.140519,100.0,0.035794
7,-2.646992,2.698357,100.0,0.149716
8,-3.230947,2.525445,100.0,0.151307


### Example: Filling Missing Values with Group-Specific Values

In [35]:
s = pd.Series(np.random.randn(6))

s[::2] = np.nan

In [36]:
s

0         NaN
1   -0.091232
2         NaN
3    0.500344
4         NaN
5   -0.221185
dtype: float64

In [37]:
s.fillna(s.mean())

0    0.062642
1   -0.091232
2    0.062642
3    0.500344
4    0.062642
5   -0.221185
dtype: float64

Suppose you need the fill value to vary by group. One way to do this is to group the
data and use apply with a function that calls fillna on each data chunk. Here is
some sample data on US states divided into eastern and western regions:

In [28]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
         'Oregon', 'Nevada', 'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4


data = pd.Series(np.random.randn(8), index=states)

In [29]:
data

Ohio         -0.509144
New York     -0.928250
Vermont       1.730833
Florida      -0.335496
Oregon        0.156598
Nevada       -0.799062
California   -0.537618
Idaho        -0.318532
dtype: float64

In [43]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan


data

Ohio          0.553903
New York     -0.450071
Vermont            NaN
Florida      -1.315241
Oregon        1.615082
Nevada             NaN
California    1.519570
Idaho              NaN
dtype: float64

In [44]:
data.groupby(group_key).mean()

East   -0.403803
West    1.567326
dtype: float64

In [45]:
# We can fill the NA values using the group means like so
fill_mean = lambda g: g.fillna(g.mean())

data.groupby(group_key).apply(fill_mean)

Ohio          0.553903
New York     -0.450071
Vermont      -0.403803
Florida      -1.315241
Oregon        1.615082
Nevada        1.567326
California    1.519570
Idaho         1.567326
dtype: float64

In [46]:
# In another case, you might have predefined fill values in your code that vary by
# group. Since the groups have a name attribute set internally, we can use that:

fill_values = {'East': 0.5, 'West': -1}

fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

Ohio          0.553903
New York     -0.450071
Vermont       0.500000
Florida      -1.315241
Oregon        1.615082
Nevada       -1.000000
California    1.519570
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation


Suppose you wanted to draw a random sample (with or without replacement) from a
large dataset for Monte Carlo simulation purposes or some other application. There
are a number of ways to perform the “draws”

In [3]:
# To demonstrate, here’s a way to construct a deck of English-style playing cards:

# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []

for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)


In [4]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [5]:
# Now, based on what I said before, drawing a hand of five cards from the deck could be written as:
def draw(deck, n=5):
     return deck.sample(n)
    
draw(deck)


10C    10
9H      9
QC     10
2D      2
10D    10
dtype: int64

In [6]:
# Suppose you wanted two random cards from each suit. Because the suit is the last
# character of each card name, we can group based on this and use apply:

get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  4C      4
   3C      3
D  KD     10
   2D      2
H  7H      7
   10H    10
S  10S    10
   8S      8
dtype: int64

In [7]:
# Alternatively, we could write:

deck.groupby(get_suit, group_keys=False).apply(draw, n=2)


7C     7
KC    10
5D     5
2D     2
KH    10
3H     3
JS    10
8S     8
dtype: int64

### Example: Group Weighted Average and Correlation

In [8]:
# Under the split-apply-combine paradigm of groupby, operations between columns in
# a DataFrame or two Series, such as a group weighted average, are possible.

df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                  'b', 'b', 'b', 'b'],
                  'data': np.random.randn(8),
                  'weights': np.random.rand(8)})

df


,category,data,weights
0,a,-0.611209,0.001991
1,a,0.590840,0.970416
2,a,-1.517715,0.345432
3,a,-0.996259,0.001998
4,b,0.175016,0.153898
5,b,-0.809533,0.316402
6,b,0.303756,0.585771
7,b,-0.251811,0.769851


In [10]:
# The group weighted average by category would then be:
grouped = df.groupby('category')

get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

grouped.apply(get_wavg)



category
a    0.034767
b   -0.134249
dtype: float64

In [12]:
close_px = pd.read_csv('C:/Users/Justin/Downloads/pydata-book-2nd-edition/examples/stock_px_2.csv', parse_dates=True, index_col=0)

close_px.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [13]:
 close_px[-4:]


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [14]:
spx_corr = lambda x: x.corrwith(x['SPX'])

rets = close_px.pct_change().dropna()

get_year = lambda x: x.year

by_year = rets.groupby(get_year)

by_year.apply(spx_corr)


,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [15]:
# You could also compute inter-column correlations. Here we compute the annual cor‐
# relation between Apple and Microsoft:

by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

 ### Example: Group-Wise Linear Regression


In [16]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params


In [17]:
# Now, to run a yearly linear regression of AAPL on SPX returns, execute:
by_year.apply(regress, 'AAPL', ['SPX'])


,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


 ### Pivot Tables and Cross-Tabulation

In [21]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [22]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')


size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [23]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                  columns='smoker', margins=True)


size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [25]:
# To use a different aggregation function, pass it to aggfunc. For example, 'count' or
# len will give you a cross-tabulation (count or frequency) of group sizes:

tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                  aggfunc=len, margins=True)


day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [26]:
#If some combinations are empty (or otherwise NA), you may wish to pass a fill_value:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)


day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Cross-Tabulations: Crosstab

In [30]:
#A cross-tabulation (or crosstab for short) is a special case of a pivot table that computes group frequencies

data

Ohio         -0.509144
New York     -0.928250
Vermont       1.730833
Florida      -0.335496
Oregon        0.156598
Nevada       -0.799062
California   -0.537618
Idaho        -0.318532
dtype: float64

In [31]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

AttributeError: 'Series' object has no attribute 'Nationality'

In [32]:
# The first two arguments to crosstab can each either be an array or Series or a list of arrays

pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)


smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244